# Defining a Data-Driven Domain of Shapes

**Input to define domain**:

- Dataset of 2D shapes (cross sections)
- Dependencies detailed below

In [1]:
# Python
import os
import numpy as np

# G2Aero
from g2aero.PGA import PGAspace, Dataset
from g2aero import Grassmann as gr

# plotting routines
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Read airfoils data from data subdirectory
First we need to load dataset of airfoils shapes stored in one of .npz files in `data/airfoils/`. 

In [2]:
shapes_folder  = os.path.join(os.getcwd(), '../../data/airfoils/', )
#shapes = np.load(os.path.join(shapes_folder, 'shapes.npz'))['shapes']
shapes = np.load(os.path.join(shapes_folder, 'CST_shapes_TE_gap.npz'))['shapes']
print("Dataset:")
print(f"Shape of data = {shapes.shape}")
print(f"N_shapes = {shapes.shape[0]}")
print(f"n_landmarks in every shape = {shapes.shape[1]}")

Dataset:
Shape of data = (13000, 401, 2)
N_shapes = 13000
n_landmarks in every shape = 401


## Build PGA space and get coordinates
Now we can construct PGA space from given dataset of airfoil shapes. `PGAspace.create_from_dataset()` method returns PGAspace object and array of airfoils coordinates in this space.

In [3]:
# compute Karcher mean and run PGA to define coordinates
pga, t = PGAspace.create_from_dataset(shapes)

Karcher mean convergence:
||V||_F = 0.10235806971182138
||V||_F = 0.00015526889897758003
||V||_F = 2.7114758181406616e-07
||V||_F = 5.985411586299064e-10


To visualize this PGA space we plot first four (out of 2*n_landmarks) coordinates of dataset shapes as scatterplot with marginal distribution on diagnal.

In [4]:
coord_names = ['$t_1$', '$t_2$', '$t_3$', '$t_4$']
df_all = pd.DataFrame(data=t[:, :4], columns=coord_names)
sns_plot = sns.pairplot(df_all, x_vars=coord_names, y_vars=coord_names,
                        diag_kind='kde', corner=True, plot_kws=dict(s=15))

## Low-dimensional shape reconstruction
Let's reconstruct a shape using fewer parameters (using lower-dimentional PGA space) and compare it to the original shape. Here we set the reduced number of parameters (the dimension of PGA space) to `r=4` and choose the shape with index `j=1` for demonstration. We also need to calculate LA standardization. LA-standardized shapes and corresponding affine transformation is stored in the `data` object of `Dataset` class.

In [5]:
# assign r as the dimension of the PGA shape
r = 4 # should always be less than or equal to 2*(n_landmarks - 2)
# pick a shape based on index from the dataset
j = 1 # should be less than or equal to N_shapes-1
# save LA standardized shapes and corresponding affine transformation
data = Dataset(shapes)

To compare original and reconstructed shape we can use Grassmannian distanse (distance measure between shape elements on Grassmannian) and worst-case Euclidean error (maximum of Euclidean distances between corresponding shape nodes). 

In [6]:
# transform from PGA space to element on Grassmann
shape_gr_new = pga.PGA2gr_shape(t[j,:r], original_shape_gr=data.shapes_gr[j])
# compute Grassmannian distance error
err_gr = gr.distance(shape_gr_new, data.shapes_gr[j])
print(f'Grassmannian distance: {err_gr}')

Grassmannian distance: 0.03607142623675289


In [7]:
def norm_inf2(sh1, sh2):
    d = np.max(np.linalg.norm(sh1-sh2, ord=2, axis=1))
    return d

# transform from PGA space to physical scales
phys_shape = pga.PGA2shape(t[j,:r], M=data.M[j], b=data.b[j], 
                           original_shape_gr=data.shapes_gr[j])
# compute worst-case Euclidean error in row-wise landmarks
err_inf2 = norm_inf2(phys_shape, shapes[j])
print(f'Worst-case Euclidean error: {err_inf2}')

Worst-case Euclidean error: 0.004075096848161961


In [8]:
# plot the low-dimensional shape and the original shape
fig, ax = plt.subplots(1, 1, figsize=(12, 5))
plt.plot(phys_shape[:,0], phys_shape[:,1],linewidth=2.5, label='low-dimensional shape')
plt.plot(shapes[j,:,0], shapes[j,:,1],'--',linewidth=2.5, label='original shape')
# formatting
plt.axis('off')
ax.axis('equal')
ax.legend(fontsize='x-large')